In [1]:
%load_ext autoreload

In [2]:
"""
Grandfather passage sentences = 
0: You wish to know about my grandfather 
1: Well he is nearly ninety three years old 
2: yet he still thinks as swiftly as ever
3: He dresses himself in an old black frock coat
4: usually several buttons missing
5: A long beard clings to his chin
6: giving those who observe him
7: a pronounced feeling of the utmost respect
8: When he speaks his voice is
9: just a bit cracked and quivers a bit
10: Twice each day he plays skillfully
11: and with zest upon a small organ
12: Except in the winter when
13: the snow or ice prevents
14: he slowly takes a short walk in
15: the open air each day
16: We have often urged him to walk
17: more and smoke less
18: but he always answers
19: Banana oil Grandfather likes 
20: to be modern in his language

Each sentence is articulated silently as one would naturally speak in English language.  
DATA is an array of (number snetences * 6, 22 - number channels, 20.000 - number time steps).
"""

'\nGrandfather passage sentences = \n0: You wish to know about my grandfather \n1: Well he is nearly ninety three years old \n2: yet he still thinks as swiftly as ever\n3: He dresses himself in an old black frock coat\n4: usually several buttons missing\n5: A long beard clings to his chin\n6: giving those who observe him\n7: a pronounced feeling of the utmost respect\n8: When he speaks his voice is\n9: just a bit cracked and quivers a bit\n10: Twice each day he plays skillfully\n11: and with zest upon a small organ\n12: Except in the winter when\n13: the snow or ice prevents\n14: he slowly takes a short walk in\n15: the open air each day\n16: We have often urged him to walk\n17: more and smoke less\n18: but he always answers\n19: Banana oil Grandfather likes \n20: to be modern in his language\n\nEach sentence is articulated silently as one would naturally speak in English language.  \nDATA is an array of (number snetences * 6, 22 - number channels, 20.000 - number time steps).\n'

In [3]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from manifoldRnn import spdNN
from manifoldRnn import optimizers 
from manifoldRnn import trainTest
from manifoldRnn import spdRnn

In [4]:
class BaseDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        return self.data[index].astype('float32'), self.labels[index]

    def __len__(self):
        return len(self.data)

In [5]:
dev = "cuda:0" 
device = torch.device(dev)

In [6]:
numberSentences = 21
trialsPerSentence = 6
numberTrials = numberSentences * trialsPerSentence
numberChannels = 22
windowLength = 5000 * 4

In [7]:
subjectNumber = 1
subject = "Subject" + str(subjectNumber)

In [8]:
DATA = np.load("Experiment2/" + subject + "/grandfatherPassageContinuousSentences.npy")
   
mean = np.mean(DATA, axis = -1)
std = np.std(DATA, axis = -1)
DATA = (DATA - mean[..., np.newaxis])/(std[..., np.newaxis] + 1e-5)

In [9]:
slicedMatrices = np.zeros((numberTrials, 36, numberChannels, numberChannels))
for j in range(numberTrials):
    for i in range(36):
        where = i * 500 + 1000
        start = where - 1000
        End = where + 1000
        slicedMatrices[j, i]= 1/2000 * (DATA[j, :, start:End] @ DATA[j, :, start:End].T)

In [10]:
labelsBySentences = np.array([[i] * trialsPerSentence for i in range(numberSentences)]).reshape(numberTrials)

Indices =  {}
for i in range(numberSentences):
    Indices[i] = []
for i in range(len(labelsBySentences)):
    Indices[labelsBySentences[i]].append(i)

covariancesLabels = np.zeros((numberSentences, trialsPerSentence, 36, numberChannels, numberChannels))
for i in range(numberSentences):
    for j in range(trialsPerSentence):
        covariancesLabels[i, j] = slicedMatrices[Indices[i][j]] 

In [11]:
trainFeatures = np.zeros((numberSentences * 4, 36, numberChannels, numberChannels))
trainLabels = np.zeros((numberSentences * 4))
count = 0
for i in range(numberSentences):
    trainFeatures[count:count + 4] = covariancesLabels[i, :4]
    trainLabels[count:count + 4] = [i] * 4
    count += 4

testFeatures = np.zeros((numberSentences * 2, 36, numberChannels, numberChannels))
testLabels = np.zeros((numberSentences * 2))
count = 0
for i in range(numberSentences):
    testFeatures[count:count + 2] = covariancesLabels[i, 4:]
    testLabels[count:count + 2] = [i] * 2
    count += 2

In [12]:
trainDataset = BaseDataset(trainFeatures, trainLabels)
testDataset = BaseDataset(testFeatures, testLabels)
trainDataloader = DataLoader(trainDataset, batch_size = 32, shuffle = True)
testDataloader = DataLoader(testDataset, batch_size = 32, shuffle = False)

In [13]:
numberEpochs = 200

model = spdRnn.spdRnnNet(numberSentences).to(device)
numParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(numParams)
lossFunction = nn.CrossEntropyLoss()
cnnOptimizer = optimizers.StiefelOptim(model.CNN.parameters(), lr = 0.05)
rnnOptimizer = optim.Adam(model.RNN.parameters(), lr = 0.001, weight_decay = 1e-3)

145368


In [14]:
maxValue = 0
for epoch in range(numberEpochs):
    trainLoss, trainAccuracy = trainTest.trainOperation(model, device, trainDataloader, cnnOptimizer, rnnOptimizer, lossFunction)
    testLoss, testAccuracy = trainTest.testOperation(model, device, testDataloader, lossFunction)
    if maxValue < testAccuracy:
        maxValue = testAccuracy
    print(f'Epoch: {epoch + 1}/{numberEpochs}, Training loss: {trainLoss:.4f}, Training accuracy: {trainAccuracy:.2f}%, Val loss: {testLoss:.4f}, Val accuracy: {testAccuracy:.2f}%')
print(maxValue)

Epoch: 1/200, Training loss: 0.1106, Training accuracy: 2.38%, Val loss: 0.1439, Val accuracy: 4.76%
Epoch: 2/200, Training loss: 0.1086, Training accuracy: 10.71%, Val loss: 0.1441, Val accuracy: 4.76%
Epoch: 3/200, Training loss: 0.1077, Training accuracy: 13.10%, Val loss: 0.1441, Val accuracy: 14.29%
Epoch: 4/200, Training loss: 0.1076, Training accuracy: 20.24%, Val loss: 0.1439, Val accuracy: 21.43%
Epoch: 5/200, Training loss: 0.1070, Training accuracy: 20.24%, Val loss: 0.1438, Val accuracy: 14.29%
Epoch: 6/200, Training loss: 0.1066, Training accuracy: 23.81%, Val loss: 0.1433, Val accuracy: 19.05%
Epoch: 7/200, Training loss: 0.1062, Training accuracy: 23.81%, Val loss: 0.1430, Val accuracy: 21.43%
Epoch: 8/200, Training loss: 0.1057, Training accuracy: 28.57%, Val loss: 0.1423, Val accuracy: 16.67%
Epoch: 9/200, Training loss: 0.1048, Training accuracy: 30.95%, Val loss: 0.1418, Val accuracy: 14.29%
Epoch: 10/200, Training loss: 0.1039, Training accuracy: 26.19%, Val loss: 0